In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold#, KFold
# from sklearn.metrics import roc_auc_score
# from fastai.vision.all import *
import wandb
# import timm

In [2]:
# approach = ''

In [3]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchinfo import summary
# # from fastai.callback.wandb import *
# from torch.cuda.amp import autocast, GradScaler

In [4]:
# from fastai.vision.all import *
from fastai.tabular.all import *
# from fastai.callback.wandb import WandbCallback

In [5]:
# import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [6]:
%config Completer.use_jedi = False

In [7]:
set_seed(42, reproducible=True)

In [8]:
datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/202108_august/')

In [9]:
df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)

In [10]:
df.isnull().sum().any()

False

In [11]:
# df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250000 entries, 0 to 249999
Data columns (total 101 columns):
 #    Column  Non-Null Count   Dtype  
---   ------  --------------   -----  
 0    f0      250000 non-null  float64
 1    f1      250000 non-null  int64  
 2    f2      250000 non-null  float64
 3    f3      250000 non-null  float64
 4    f4      250000 non-null  float64
 5    f5      250000 non-null  float64
 6    f6      250000 non-null  float64
 7    f7      250000 non-null  float64
 8    f8      250000 non-null  float64
 9    f9      250000 non-null  float64
 10   f10     250000 non-null  float64
 11   f11     250000 non-null  float64
 12   f12     250000 non-null  float64
 13   f13     250000 non-null  float64
 14   f14     250000 non-null  float64
 15   f15     250000 non-null  float64
 16   f16     250000 non-null  int64  
 17   f17     250000 non-null  float64
 18   f18     250000 non-null  float64
 19   f19     250000 non-null  float64
 20   f20     250000 non-null 

<ipython-input-11-6ac79f7ef903>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose=True, null_counts=True)


So no NaNs.

In [12]:
# df.describe()

In [13]:
y = df.loss

In [14]:
features = [x for x in df.columns if x != 'loss']

In [15]:
X = df[features]

In [16]:
len(X)

250000

In [17]:
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,1.09101,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,1.28138,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,1.24210,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,1.09130,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,1.10151,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [18]:
len(y), len(y.value_counts())

(250000, 43)

So, it seems that we're actually looking at a classification problem here -- not a regression one.

# RandomForestRegressor with some hyperparameter tuning

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.ensemble import RandomForestRegressor

In [47]:
model = RandomForestRegressor(random_state=42, min_samples_leaf=2, n_jobs=-1, n_estimators=100)

In [48]:
# import wandb
wandb.init(
    project="202108_Kaggle_tabular_playground",
    save_code=True,
    tags=['RandomForest'],
    name='202108061912_RandomForestRegressor',
    notes='Changing some hyperparameters',
    config={
        'library': 'scikit-learn',
        'model': model,
        'params': model.get_params()
    }
        
    #                'initial_learning_rate':lr_steep,
#        'metric':'roc_auc',
#        'n_epochs': n_epochs,
#        'batch_size': batch_size,
#        'optimizer':optimizer,
#        'loss_func':loss_func,
#        'mixup_applied': mixup_applied,
#        'mixup_alpha': mixup_alpha,
#        'architecture':arch,
#        'model_source': model_source,
#        'pretrained': pretrained,
#        'transform_types': transform_types,
#        'transform_libraries': transform_libraries,
#        'cv_transforms': cv_transforms_applied,
#        'dsp_transforms': dsp_transforms_applied,
#        'trainset_type':trainset_type,
#        'k-fold': fold,
#        'k-fold_total': k_folds,
#        'k-fold_strategy': 'stratified',
#        'lr_strategy': 'valley',
#        'lr': lr
)

wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [49]:
model.fit(X_train, y_train)
# model.fit(X, y)

RandomForestRegressor(min_samples_leaf=2, n_jobs=-1, random_state=42)

In [50]:
wandb.sklearn.plot_regressor(model, X_train, X_valid, y_train, y_valid,  model_name='RandomForestRegressor', )

wandb: 
wandb: Plotting RandomForestRegressor.
wandb: Logged summary metrics.
wandb: Logged learning curve.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged outlier candidates.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged residuals.


In [ ]:
# y_pred = model.predict(X_valid)

In [28]:
from sklearn.model_selection import cross_val_score

In [ ]:
# scores = cross_val_score(model, X, y, cv=5, n_jobs=-1)

In [31]:
# model.get_depth()

In [56]:
from sklearn.metrics import mean_squared_error

In [57]:
y_valid_preds = model.predict(X_valid)

In [58]:
mse = mean_squared_error(y_valid, y_valid_preds)

In [59]:
mse

63.145484029282386

In [60]:
rmse = math.sqrt(mse)
rmse

7.946413280800488

In [44]:
# rmses = [math.sqrt(abs(score)) for score in scores]
# rmses

Let's give that a submit...

In [37]:
test_df = pd.read_csv(datapath/'test.csv', index_col='id', low_memory=False)

In [38]:
test_df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,
250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,1.11394,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,1.09695,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,1.15222,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,1.20157,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,1.16807,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [39]:
X_test = test_df[features] # this is just for naming consistency

In [40]:
y_test_preds = model.predict(X_test)

In [41]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [42]:
sample_df.loc[:, 'loss'] = y_test_preds

In [43]:
sample_df.head()

,id,loss
0,250000,5.066757
1,250001,7.624070
2,250002,6.684094
3,250003,5.387886
4,250004,8.677814


In [45]:
sample_df.to_csv('202108061645_RandomForestRegressor.csv', index=False)

In [61]:
wandb.finish()

_runtime,4710
_timestamp,1628307084
_step,3


_runtime,▁▇██
_timestamp,▁▇██
_step,▁▃▆█
